# Simulations light inter

In [1]:
from __future__ import annotations

import os
import sys
import time as t
from datetime import date
from multiprocessing import Pool

import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

sys.path.append("../data") 
from archi_dict import archi_sorghum_angles as archi
from oawidgets.plantgl import *  # noqa: F403

from openalea.archicrop.display import build_scene, display_scene  # noqa: F401
from openalea.archicrop.simulation import (
    plot_constained_vs_pot,
    plot_constrainted_vs_realized,
    plot_faPAR,
    plot_PAR,
    run_simulations,
    write_netcdf,
)
from openalea.plantgl.all import Color3, Material, Scene  # noqa: F401

%gui qt

In [2]:
stics_color = "orange"
archicrop_color = "green"

In [3]:
tec_file_xml='../data/02NT18SorgV2D1_tec.xml'
plt_file_xml='../data/sorgho_imp_M_v10_plt.xml'
stics_output_file='../data/mod_s02NT18SorgV2D1.sti'
weather_file = '../data/ntarla_corr.2018'
location = {  
'longitude': 3.87,
'latitude': 12.58,
'altitude': 800,
'timezone': 'Europe/Paris'}

In [4]:
seed = 18

In [5]:

start_time = t.time()

daily_dynamics, params_sets, pot_la, pot_h, realized_la, realized_h, nrj_per_plant, mtgs, filters, sowing_density = run_simulations(
    archi_params=archi, 
    tec_file=tec_file_xml, 
    plant_file=plt_file_xml, 
    dynamics_file=stics_output_file, 
    weather_file=weather_file,
    location=location,
    n_samples=3,
    latin_hypercube=False,
    opt_filter_organ_duration=False,
    opt_filter_pot_growth=False,
    opt_filter_realized_growth=False,
    light_inter=True,
    direct=False,
    error_LA_pot=1,
    error_height_pot=1,
    error_LA_realized=1,
    error_height_realized=1,
    seed=seed)

end_time = t.time()

elapsed_time = (end_time - start_time)/60
print(f"Elapsed time: {elapsed_time:.4f} minutes for {len(realized_la)} simulations")  # noqa: T201


Elapsed time: 465.7501 minutes for 27 simulations


In [6]:
# Retrieve STICS growth and senescence dynamics
dates = [value["Date"] for value in daily_dynamics.values()]
thermal_time = [value["Thermal time"] for value in daily_dynamics.values()]
leaf_area_plant = [value["Plant leaf area"] for value in daily_dynamics.values()]
sen_leaf_area_plant = [value["Plant senescent leaf area"] for value in daily_dynamics.values()]
height_canopy = [value["Plant height"] for value in daily_dynamics.values()]
par_incident = [value["Incident PAR"] for value in daily_dynamics.values()]
par_stics = [value["Absorbed PAR"] for value in daily_dynamics.values()]

In [7]:
write_netcdf("results_light_inter", daily_dynamics, params_sets, pot_la, pot_h, realized_la, realized_h, nrj_per_plant, mtgs, filters, sowing_density, seed)

In [37]:
from openalea.plantgl.all import Color3, Material, Scene, Viewer
today_str = date.today().strftime("%Y-%m-%d")
os.makedirs(f"D:/PhD_Oriane/simulations_ArchiCrop/{today_str}", exist_ok=True)  # noqa: PTH103
# plt.savefig(f"D:/PhD_Oriane/simulations_ArchiCrop/{today_str}/plot_extin_coef_LAI_{parameter_name}.png")
times = [31, 41, 70]
for k,mtg in mtgs.items():
    for time in times:
        Viewer.display(build_scene(mtg[time])[0])
        Viewer.grids.setXYPlane(False)
        Viewer.grids.setYZPlane(False)
        Viewer.grids.setXZPlane(False)
        Viewer.camera.lookAt((386.117,-232.613,435.8), (0,0,100))
        Viewer.frameGL.saveImage(f"D:/PhD_Oriane/simulations_ArchiCrop/{today_str}/sorghum_{k}_{time}.png")

In [38]:
from openalea.plantgl.all import Color3, Material, Scene, Viewer
from openalea.archicrop.stand import agronomic_plot
%gui qt

n_emerged, positions, domain, domain_area, _ = agronomic_plot(
    length=10,
    width=10,
    sowing_density=sowing_density,
    inter_row=0.7,
    noise=0,
    convunit=100,
    center_scene=True)

# print(len(positions))

scene = build_scene([mtgs[13][70]]*len(positions), positions)[0]
Viewer.display(scene)
# Viewer.grids.setXYPlane(False)
# Viewer.grids.setYZPlane(False)
# Viewer.grids.setXZPlane(False)
# Viewer.camera.lookAt((386.117,-232.613,435.8), (0,0,100))
# Viewer.frameGL.saveImage(f"D:/PhD_Oriane/simulations_ArchiCrop/{today_str}/sorghum_{k}_{time}.png")

In [21]:
params_sets

{0: {'nb_short_phy': 5,
  'short_phy_height': 3,
  'height': 178.3812,
  'stem_q': 0.9,
  'diam_base': 2.2,
  'diam_top': 1.2,
  'leaf_area': 3884.76,
  'rmax': 0.67,
  'skew': 0.0001,
  'wl': 0.12,
  'klig': 0.6,
  'swmax': 0.55,
  'f1': 0.64,
  'f2': 0.92,
  'scurv': 0.7,
  'phyllotactic_deviation': 0,
  'phyllochron': 54,
  'stem_duration': 1.6,
  'leaf_duration': 1.6,
  'nb_tillers': 0,
  'tiller_angle': 20,
  'tiller_delay': 1,
  'reduction_factor': 1,
  'leaf_lifespan': 280.0,
  'insertion_angle': 10.0,
  'curvature': 0.0,
  'phyllotactic_angle': 90.0},
 1: {'nb_short_phy': 5,
  'short_phy_height': 3,
  'height': 178.3812,
  'stem_q': 0.9,
  'diam_base': 2.2,
  'diam_top': 1.2,
  'leaf_area': 3884.76,
  'rmax': 0.67,
  'skew': 0.0001,
  'wl': 0.12,
  'klig': 0.6,
  'swmax': 0.55,
  'f1': 0.64,
  'f2': 0.92,
  'scurv': 0.7,
  'phyllotactic_deviation': 0,
  'phyllochron': 54,
  'stem_duration': 1.6,
  'leaf_duration': 1.6,
  'nb_tillers': 0,
  'tiller_angle': 20,
  'tiller_delay': 